In [1]:
import numpy as np
import cv2
import os
from loader import DatasetGenerator
import pandas as pd

import keras
import tensorflow as tf
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
train_data = np.load('train_data.npy', allow_pickle=True)

In [4]:
input_1 = keras.layers.Input(shape=(128,))
input_2 = keras.layers.Input(shape=(128,))
concat = keras.layers.concatenate([input_1, input_2])
x = keras.layers.Dense(units=64, activation='relu')(concat)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(units=32, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(units=16, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
out = keras.layers.Dense(units=1, activation='sigmoid')(x)
model = keras.Model(inputs=[input_1, input_2], outputs=out)

In [5]:
model.compile(optimizer='sgd', loss='binary_crossentropy')

In [6]:
class Generator:
    def __init__(self, data):
        self.n = len(data)
        self.data = data
    
    def two_random(self, n):
        first = np.random.randint(n)
        second = np.random.randint(n)
        while second == first:
            second = np.random.randint(n)
        return first, second
    
    def generate_positive(self, size):
        first_part = []
        second_part = []
        for i in range(size):
            idx = np.random.randint(self.n)
            first, second = self.two_random(len(self.data[idx]))
            first_part.append(self.data[idx][first])
            second_part.append(self.data[idx][second])
        return np.array(first_part), np.array(second_part), np.ones(size)
    
    def generate_negative(self, size):
        first_part = []
        second_part = []
        for i in range(size):
            first, second = self.two_random(self.n)
            first_idx = np.random.randint(len(self.data[first]))
            second_idx = np.random.randint(len(self.data[second]))
            first_part.append(self.data[first][first_idx])
            second_part.append(self.data[second][second_idx])
        return np.array(first_part), np.array(second_part), np.zeros(size)
    
    def get_batch(self, batch_size):
        half_size = int(batch_size / 2)
        pos_first, pos_second, pos_labels = self.generate_positive(half_size)
        neg_first, neg_second, neg_labels = self.generate_negative(half_size)
        return [np.concatenate([pos_first, neg_first]), np.concatenate([pos_second, neg_second])], np.concatenate([pos_labels, neg_labels])

In [7]:
generator = Generator(train_data)

In [87]:
epochs = 50000
steps = 1000
batch_size = 64
with tqdm(range(epochs), desc='ML') as t:
    for epoch in t:
        losses = []
        for step in tqdm(range(steps), leave = False):
            x, y = generator.get_batch(batch_size)
            loss = model.train_on_batch(x, y)
            losses.append(loss)
        t.set_description('ML (loss=%g)' % np.average(losses))

KeyboardInterrupt: 

In [12]:
test_data = np.load('test_data.npy', allow_pickle=True)

In [34]:
mapping = pd.read_csv(r'class_mapping.csv')
names = np.array(mapping['Id'])
test_labels = np.array(mapping['Category'])
test_folder = r'C:/Datasets/test_data/test_data/'
test_images = os.listdir(test_folder)

In [67]:
name_to_label = {}
for i in range(len(names)):
    name_to_label[names[i]] = test_labels[i]

In [68]:
def get_idx(name):
    name = int(name.split('.')[0])
    return name

In [72]:
labeled_data = []
labeled_idx = []
for name in names:
    idx = get_idx(name)
    labeled_data.append(test_data[idx])
labeled_data = np.array(labeled_data)

In [73]:
labeled_data.shape

(500, 128)

In [77]:
labels = np.argmax(np.dot(test_data, labeled_data.T), axis=1)
labels = test_labels[labels]

In [88]:
labels = np.zeros(len(test_data))
values = np.zeros(len(test_data))
for vec_id, vector in enumerate(tqdm(test_data)): 
    tiled = np.tile(vector, len(labeled_data)).reshape(-1, len(vector))
    prediction = np.argmax(model.predict([labeled_data, tiled]))
    labels[vec_id] = test_labels[prediction]

In [89]:
result_dict = {'Id':[], 'Random':[]}
for idx, image in tqdm(enumerate(test_data)):
    image_name = test_images[idx]
    result_dict['Id'].append(image_name)
    result_dict['Random'].append(int(labels[idx]))

In [90]:
result_frame = pd.DataFrame(result_dict)

In [91]:
result_frame

,Id,Random
0,000000.jpg,0
1,000001.jpg,1
2,000002.jpg,2
3,000003.jpg,3
4,000004.jpg,209
5,000005.jpg,395
6,000006.jpg,28
7,000007.jpg,7
8,000008.jpg,8
9,000009.jpg,9


In [80]:
result_frame

,Id,Random
0,000000.jpg,0
1,000001.jpg,1
2,000002.jpg,2
3,000003.jpg,3
4,000004.jpg,73
5,000005.jpg,395
6,000006.jpg,6
7,000007.jpg,7
8,000008.jpg,8
9,000009.jpg,9


In [92]:
result_frame.to_csv('D:/result.csv', index=False)